# 2-step overviews with pre-scan

Here, we perform imaging of overviews in a regular grid as in other notebooks. But in addition, we do a **quick pre-scan** with larger pixel size at each location. Only at tiles where we see at least some signal, we do the actual (slower) imaging.

In [1]:
import specpy

from autosted.callback_buildingblocks import (
    FOVSettingsGenerator,
    JSONSettingsLoader,
    LocationKeeper,
    LocationRemover,
    NewestSettingsSelector,
    PositionListOffsetGenerator,
    ScanModeSettingsGenerator,
)
from autosted.detection import AcceptanceCheck
from autosted.imspector import get_current_stage_coords
from autosted.pipeline import AcquisitionPipeline
from autosted.taskgeneration import AcquisitionTaskGenerator
from autosted.utils.tiling import centered_tiles

We will make use of the ```AcceptanceCheck``` in the pipeline. For that, we need to define a function that returns a boolean (image here / don't image here) decision given an image:

In [2]:
def area_above_threshold_check(img, threshold=10, min_area=25):
    """simple check if enough pixels above background are in an image"""
    return (img >= threshold).sum() >= min_area

To test the function, we can e.g. get the currently open image from Imspector:

In [ ]:
# get current image from Imspector to test
im = specpy.get_application()
ms = im.active_measurement()
img = ms.stack(0).data().squeeze()

area_above_threshold_check(img, threshold=100)

## Parameters of the acquisition

In [11]:
# where to save & whether to save combined HDF5 file
save_folder = "acquisition_data/20241010_selective_overviews"
save_hdf5 = False

# path of measurement parameters (dumped to JSON file)
measurement_parameters = "examples/config_json/20241010_overview_3d_640.json"

# yx FOV size
fov_size = [50e-6, 50e-6]

# (larger) pixel sizes for pre_scan (e.g. 1micron)
pre_scan_pixelsizes = [1e-6, 1e-6, 1e-6]

# scan mode of pre-scan
# 'xy' to only do one plane
pre_scan_mode = "xy"

# parameter values for area_above_threshold_check
acceptance_parameters = {"threshold": 50, "min_area": 50}

# yx number of tiles
n_tiles = [5, 5]

# how much the tiles should overlap (0-1)
overlap_fraction = 0

In [ ]:
# get current coordinates and print, so we can go back to that position
start_coords = get_current_stage_coords()
print(start_coords)

# generate regular grid around current stage position
coordinate_list = centered_tiles(
    start_coords,
    fov_size=[0] + fov_size,
    n_tiles=[1] + n_tiles,
    overlap=overlap_fraction,
)

# show for verification
coordinate_list

## Setup and run pipeline

Now, we set up a two-level ("pre_scan", "field") pipeline. The pre-scans are done in the regular grid defined above. Additionally in the callback generating them (```next_position_generator```), we use a ```ScanModeSettingsGenerator``` to e.g. only image one xy plane and a ```FOVSettingsGenerator``` to set a larger pixel size.

Additionally, we use a second callback (```accept_position_callback```) to enqueue proper images (smaller pixel size, 3D stack) only at positions where our AcceptanceCheck is True.

In [12]:
# build pipeline object (2 levels: 'pre_scan', 'field')
pipeline = AcquisitionPipeline(
    save_folder,
    ["pre_scan", "field"],
    save_combined_hdf5=save_hdf5,
    name="multipoint-acquisition",
)

# NOTE: some internal index / filename handling
# field will re-use the index of the associated pre-scan
# e.g. if we image after pre-scan (2,) instead of (2, 0), we give the field idx (2, 2)
pipeline._levels_reusing_parent_index = ["field"]
# pre-scan index will not show up in filename (only for actual pre-scan images)
pipeline._masked_levels_in_filename = ["pre_scan"]

# callback that will create an acquisition task with given measurement parameters
# at the next stage coordinates in the coordinate list (the next 'position')
next_position_generator = AcquisitionTaskGenerator(
    "pre_scan",
    LocationRemover(JSONSettingsLoader(measurement_parameters)),
    # set different scan mode (e.g. only "xy")
    ScanModeSettingsGenerator(pre_scan_mode),
    # set different pixel size
    FOVSettingsGenerator(pixel_sizes=pre_scan_pixelsizes),
    PositionListOffsetGenerator(coordinate_list),
)

# callback to create a measurement with same location, but larger pixel size / 3D
# if something is detected in pre-scan via AcceptanceCheck
accept_position_callback = AcquisitionTaskGenerator(
    "field",
    LocationRemover(JSONSettingsLoader(measurement_parameters)),
    # re-use location of pre-scan
    LocationKeeper(NewestSettingsSelector()),
    AcceptanceCheck(
        area_above_threshold_check, check_function_kwargs=acceptance_parameters
    ),
)

# attach callback so that after each position, pre_scan the next one will be enqueued
pipeline.add_callback(next_position_generator, "pre_scan")

# attach callback so that after each pre-scan, a more detailled stack will be images, if accepted
pipeline.add_callback(accept_position_callback, "pre_scan")

# start with initial task from callback
pipeline.run(next_position_generator)